In [1]:
from building_depot import DataService, BDError
import os
import json
import arrow
import pandas as pd
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [30]:
tz_str = 'US/Pacific'

with open("config/bd2_1config.json", "r") as fp:
    configs = json.load(fp)
    hostname = configs['hostname']
    valid_naes = configs['valid_naes']
    base_dir = configs['base_dir']
    begin_time = arrow.get(configs['begin_time']).replace(tzinfo=tz_str).datetime
    end_time = arrow.get(configs['end_time']).replace(tzinfo=tz_str).datetime
with open("config/bd2_1credential.json", "r") as fp:
    cred = json.load(fp)
    username = cred['bdUserName']
    apikey = cred['bdApiKey']
ds = DataService(hostname, apikey, username)

with open("metadata/bacnet_devices.json", "r") as fp:
    bacnet_devices = json.load(fp)

In [19]:
a = begin

TypeError: astimezone() argument 1 must be datetime.tzinfo, not str

In [15]:
uuid_dict = dict()
i = 0
for nae_name in valid_naes:
    objs = bacnet_devices[nae_name]['objs']
    for obj in objs:
        if obj['data_type'] not in [0,1,2,3,4,5,13,14,19]:
            continue        
        if i%500==0:
            print i
        i += 1
        props = obj['props']
        tag_val_dict = {
            'BACNet_DataType': obj['data_type'],
            'BACNet_InstanceNumber':obj['props']['instance'],
            'NAENum':nae_name
            }
        srcid = str(nae_name) + "_" + str(obj['data_type']) + "_" + str(obj['props']['instance'])
        #try:
        resp = ds.list_sensors({'source_identifier':srcid})
        uuid = resp['sensors'][0]['uuid']
        uuid_dict[srcid] = uuid
        
        #except:
        #    print '----------------'
        #    print srcid
        #    print sensor_api.search(tag_val_dict)        

0
500
1000
1500
2000
2500
3000
3500
4000
4500


In [21]:
#isoBegin = self.pst.localize(beginTime)
#isoEnd = self.pst.localize(endTime)
rawData = ds.get_timeseries_datapoints(uuid, 'PresentValue', begin_time, end_time)
#ts = bdm.get_sensor_ts(uuid, 'PresentValue', beginTime, endTime)

In [24]:
rawData

{u'span': {u'begin': u'2014-06-24T15:55:03+00:00',
  u'end': u'2017-01-27T01:53:42+00:00'},
 u'timeseries': [{u'2017-01-01T08:00:43+00:00': 1.0},
  {u'2017-01-01T08:05:02+00:00': 1.0},
  {u'2017-01-01T08:09:21+00:00': 1.0},
  {u'2017-01-01T08:13:39+00:00': 1.0},
  {u'2017-01-01T08:18:02+00:00': 1.0},
  {u'2017-01-01T08:22:24+00:00': 1.0},
  {u'2017-01-01T08:26:45+00:00': 1.0},
  {u'2017-01-01T08:31:03+00:00': 1.0},
  {u'2017-01-01T08:35:27+00:00': 1.0},
  {u'2017-01-01T08:39:46+00:00': 1.0},
  {u'2017-01-01T08:44:09+00:00': 1.0},
  {u'2017-01-01T08:48:25+00:00': 1.0},
  {u'2017-01-01T08:52:45+00:00': 1.0},
  {u'2017-01-01T08:57:00+00:00': 1.0},
  {u'2017-01-01T09:01:21+00:00': 1.0},
  {u'2017-01-01T09:05:48+00:00': 1.0},
  {u'2017-01-01T09:10:06+00:00': 1.0},
  {u'2017-01-01T09:14:30+00:00': 1.0},
  {u'2017-01-01T09:18:46+00:00': 1.0},
  {u'2017-01-01T09:23:02+00:00': 1.0},
  {u'2017-01-01T09:27:22+00:00': 1.0},
  {u'2017-01-01T09:31:46+00:00': 1.0},
  {u'2017-01-01T09:36:12+00:00': 1.

In [31]:
header = ['value']

index_label="time"
for srcid, uuid in uuid_dict.items():
    #try:
    ts = ds.get_timeseries_datapoints(uuid, 'PresentValue', begin_time, end_time)['timeseries']
    time_list = list()
    value_list = list()
    for item in ts:
        time_list.append(item.keys()[0])
        value_list.append(item.values()[0])
    filename = base_dir + "/" + srcid + ".csv"
    pd.Series(index=time_list, data=value_list).to_csv(filename, header=header, index_label=index_label)
    #except:
    #    print srcid, uuid
    #    print ts_api.getTimeseriesDataPoints(uuid, begin_time, end_time)